![title](img/dask.png)

#### **Dask provides advanced parallelism for analytics, enabling performance at scale for the tools you love**

#### **Dask is composed of two parts:**

    1. Dynamic task scheduling optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
    2. “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.
    
[detailed information here](https://docs.dask.org/en/latest/)

#### **DASK with all its features:**
    • Built in Python
    • Scales properly from single laptops to 1000-node clusters
    • Leverages and interops with existing Python APIs as much as possible
    • Adheres to (Tim Peters') "Zen of Python" (https://www.python.org/dev/peps/pep-0020/) ... especially these elements:
        ◦ Explicit is better than implicit.
        ◦ Simple is better than complex.
        ◦ Complex is better than complicated.
        ◦ Readability counts. [ed: that goes for docs, too!]
        ◦ Special cases aren't special enough to break the rules.
        ◦ Although practicality beats purity.
        ◦ In the face of ambiguity, refuse the temptation to guess.
        ◦ If the implementation is hard to explain, it's a bad idea.
        ◦ If the implementation is easy to explain, it may be a good idea.
    • While we're borrowing inspiration, it Dask embodies one of Perl's slogans, making easy things easy and hard things possible
        ◦ Specifically, it supports common data-parallel abstractions like Pandas and Numpy
        ◦ But also allows scheduling arbitary custom computation that doesn't fit a preset mold
   

    


#### **Dask emphasizes the following virtues:**


    • Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
    • Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
    • Native: Enables distributed computing in pure Python with access to the PyData stack.
    • Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
    • Scales up: Runs resiliently on clusters with 1000s of cores
    • Scales down: Trivial to set up and run on a laptop in a single process
    • Responsive: Designed with interactive computing in mind, it provides rapid feedback and diagnostics to aid humans.

In [1]:
import dask.dataframe as dd

ddf = dd.read_csv('/home/koustav/Documents/DataframeWar/Data/yellow_tripdata_2020-01.csv',
                       dtype={'RatecodeID': 'float64', 'VendorID': 'float64', 'passenger_count':'float64', 'payment_type': 'float64'},assume_missing=True, blocksize=12e6)

In [2]:
ddf.loc[0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=50,,,,,,,,,,,,,,,,,,
,float64,object,object,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
len(ddf)

6405008

In [4]:
ddf_part = dd.read_csv('/home/koustav/Documents/DataframeWar/Data/yellow_tripdata_2020-01.csv',
                       dtype={'RatecodeID': 'float64', 'VendorID': 'float64', 'passenger_count':'float64', 'payment_type': 'float64'},assume_missing=True)
ddf_part = ddf_part.repartition(npartitions=100)

In [5]:
len(ddf_part)

/home/koustav/Documents/DataframeWar/env/lib/python3.8/site-packages/dask/core.py:121: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


6405008

In [6]:
ddf_part.loc[0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=100,,,,,,,,,,,,,,,,,,
,float64,object,object,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


![title](img/dask-dataframe.png)

In [11]:
import dask.dataframe as dd

In [12]:
ddf = dd.read_csv()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=50,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,int64,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
ddf.map_partitions(type).compute(assume_missing=True)


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------------+---------+----------+
| Column          | Found   | Expected |
+-----------------+---------+----------+
| RatecodeID      | float64 | int64    |
| VendorID        | float64 | int64    |
| passenger_count | float64 | int64    |
| payment_type    | float64 | int64    |
+-----------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [ ]:
ddf.head()


In [ ]:
ddf['VendorID'].unique()

In [9]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
